In [ ]:
import statistics
import numpy as np
import pandas as pd
data = pd.read_csv('train.csv', sep=',')
data = pd.get_dummies(data, dtype='float64')
X = data.drop(['charges', 'id'], axis=1)
y = data['charges']
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True, random_state=43)
from sklearn.ensemble import RandomForestRegressor
rfr = RandomForestRegressor(random_state=49)
rfr.fit(X_train, y_train)
y_pred = rfr.predict(X_test)
from sklearn.metrics import root_mean_squared_log_error
print(root_mean_squared_log_error(y_test, y_pred))
import optuna
from sklearn.model_selection import cross_val_score
def objective(trial):
    n_estimators = trial.suggest_int('n_estimators', 100, 1500)
    criterion = trial.suggest_categorical('criterion', ['absolute_error'])
    max_depth = trial.suggest_int('max_depth', 10, 279)
    min_samples_split = trial.suggest_int('min_samples_split', 2, 49)
    min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 33)
    max_features = trial.suggest_int('max_features', 1, 11)
    bootstrap = trial.suggest_categorical('bootstrap', [True, False])
    params = dict(n_estimators=n_estimators, criterion=criterion, max_depth=max_depth, min_samples_split=min_samples_split,
                                 min_samples_leaf=min_samples_leaf, max_features=max_features,
                                 bootstrap = bootstrap,  n_jobs=4)
    if bootstrap == True:
         params['max_samples'] = trial.suggest_float("max_samples", 0.2, 1.0)
         params['oob_score'] = True
    model = RandomForestRegressor(**params)
    score = cross_val_score(model, X, y, cv=10, scoring='neg_root_mean_squared_log_error')
    return abs(statistics.mean(score))
study = optuna.create_study(direction='minimize', sampler=optuna.samplers.RandomSampler(seed=11))
study.optimize(objective, n_trials=2883)